In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
def load_precalc_params_small():

    file = open('./data/small_movies_X.csv', 'rb')
    X = np.loadtxt(file, delimiter = ",")

    file = open('./data/small_movies_W.csv', 'rb')
    W = np.loadtxt(file,delimiter = ",")

    file = open('./data/small_movies_b.csv', 'rb')
    b = np.loadtxt(file,delimiter = ",")
    b = b.reshape(1,-1)
    num_movies, num_features = X.shape
    num_users,_ = W.shape
    return(X, W, b, num_movies, num_features, num_users)
    
def load_ratings_small():
    file = open('./data/small_movies_Y.csv', 'rb')
    Y = np.loadtxt(file,delimiter = ",")

    file = open('./data/small_movies_R.csv', 'rb')
    R = np.loadtxt(file,delimiter = ",")
    return(Y,R)

In [3]:
x, w, b, movieCount, featureCount, userCount = load_precalc_params_small()
y, r = load_ratings_small()

print(f"movieCount: {movieCount}, featureCount: {featureCount}, userCount: {userCount}")
print(f"x.shape: {x.shape}")
print(f"w.shape: {w.shape}")
print(f"b.shape: {b.shape}")
print(f"y.shape: {y.shape}")
print(f"r.shape: {r.shape}")

movieCount: 4778, featureCount: 10, userCount: 443
x.shape: (4778, 10)
w.shape: (443, 10)
b.shape: (1, 443)
y.shape: (4778, 443)
r.shape: (4778, 443)


### Collaborative filtering cost function

$$J({\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},\mathbf{w}^{(0)},b^{(0)},...,\mathbf{w}^{(n_u-1)},b^{(n_u-1)}})= \left[ \frac{1}{2}\sum_{(i,j):r(i,j)=1}(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 \right]
+ \underbrace{\left[
\frac{\lambda}{2}
\sum_{j=0}^{n_u-1}\sum_{k=0}^{n-1}(\mathbf{w}^{(j)}_k)^2
+ \frac{\lambda}{2}\sum_{i=0}^{n_m-1}\sum_{k=0}^{n-1}(\mathbf{x}_k^{(i)})^2
\right]}_{regularization}
\tag{1}$$
如果有評論的話 $r$ 會標記 $1$，因此可以改寫為 "對於所有 $i$, $j$ 的 $r(i,j)$ 等於 $1$" :

$$
= \left[ \frac{1}{2}\sum_{j=0}^{n_u-1} \sum_{i=0}^{n_m-1}r(i,j)*(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 \right]
+\text{regularization}
$$

In [4]:
def computeCost(x, y, w, b, r, lambda_):
    cost = (tf.linalg.matmul(x, tf.transpose(w)) + b - y) * r
    cost = 0.5 * (tf.reduce_sum(cost ** 2) + lambda_ * (tf.reduce_sum(w ** 2) + tf.reduce_sum(x ** 2)))
    return cost

In [5]:
myRatings = np.zeros(movieCount)        

myRatings[2700] = 5 
myRatings[2609] = 2
myRatings[929]  = 5   
myRatings[246]  = 5   
myRatings[2716] = 3   
myRatings[1150] = 5   
myRatings[382]  = 2   
myRatings[366]  = 5  
myRatings[622]  = 5  
myRatings[988]  = 3   
myRatings[2925] = 1   
myRatings[2937] = 1   
myRatings[793]  = 5   

myRated = np.where(myRatings > 0, 1 ,0)

In [6]:
y = np.c_[myRatings, y]
r = np.c_[myRated, r]

yMean = (np.sum(y, axis = 1) / np.sum(r, axis = 1)).reshape(-1, 1)
yNorm = y - yMean * r

In [7]:
movieCount, userCount = y.shape
featureCount = 100

tf.random.set_seed(1234)
w = tf.Variable(tf.random.normal((userCount,  featureCount), dtype = tf.float64),  name='w')
x = tf.Variable(tf.random.normal((movieCount, featureCount), dtype = tf.float64),  name='x')
b = tf.Variable(tf.random.normal((1,          userCount),    dtype = tf.float64),  name='b')

optimizer = keras.optimizers.Adam(1e-1)

In [8]:
iterCount = 200
lambda_ = 1
for i in range(iterCount):
    with tf.GradientTape() as tape:
        cost = computeCost(x, yNorm, w, b, r, lambda_)
    grads = tape.gradient(cost, [x, w, b])
    optimizer.apply_gradients(zip(grads, [x, w, b]))
    if i % (iterCount / 10) == 0:
        print(f"Training loss at iteration {i}: {cost:0.1f}")

Training loss at iteration 0: 2321191.3
Training loss at iteration 20: 136169.3
Training loss at iteration 40: 51863.7
Training loss at iteration 60: 24599.0
Training loss at iteration 80: 13630.6
Training loss at iteration 100: 8487.7
Training loss at iteration 120: 5807.8
Training loss at iteration 140: 4311.6
Training loss at iteration 160: 3435.3
Training loss at iteration 180: 2902.1


In [9]:
p = np.matmul(x.numpy(), w.numpy().T) + b.numpy() + yMean
pred = p[:, 0]

ix = tf.argsort(pred, direction = 'DESCENDING')

for i in range(10):
    j = ix[i]
    print(f'Predicting rating {pred[j]:0.2f} for movie {j}')

print('\nOriginal vs Predicted ratings:\n')
for i in range(len(myRated)):
    if myRated[i] > 0:
        print(f'Original {myRatings[i]}, Predicted {pred[i]:0.2f} for movie {i}')

Predicting rating 4.90 for movie 1150
Predicting rating 4.90 for movie 246
Predicting rating 4.89 for movie 929
Predicting rating 4.88 for movie 622
Predicting rating 4.87 for movie 793
Predicting rating 4.84 for movie 366
Predicting rating 4.80 for movie 2700
Predicting rating 4.49 for movie 1201
Predicting rating 4.48 for movie 549
Predicting rating 4.48 for movie 211

Original vs Predicted ratings:

Original 5.0, Predicted 4.90 for movie 246
Original 5.0, Predicted 4.84 for movie 366
Original 2.0, Predicted 2.13 for movie 382
Original 5.0, Predicted 4.88 for movie 622
Original 5.0, Predicted 4.87 for movie 793
Original 5.0, Predicted 4.89 for movie 929
Original 3.0, Predicted 3.00 for movie 988
Original 5.0, Predicted 4.90 for movie 1150
Original 2.0, Predicted 2.11 for movie 2609
Original 5.0, Predicted 4.80 for movie 2700
Original 3.0, Predicted 3.00 for movie 2716
Original 1.0, Predicted 1.41 for movie 2925
Original 1.0, Predicted 1.26 for movie 2937
